In [7]:

import pandas as pd



In [76]:
df_m_0 = "min_0"
df_m_1 = "min_1"
df_m_2 = 'min_2'
# df_m_3 = pd.read_csv('../dataset/min_3.csv')
df_0 = 'informative_cluster_0'
df_1 = 'informative_cluster_1'

df_merged = 'merged_shuffled'
df_merged_labeled = 'merged_shuffled_labeled'

df_m = "df_min"
df_majority = "df_majority"

df_m_0_df_0 ="min_0_informative_cluster_0"
df_m_1_df_0 ="min_1_informative_cluster_0"
df_m_0_df_1 ="min_0_informative_cluster_1"
df_m_1_df_1 ="min_1_informative_cluster_1"
df_m_2_df_1 ="min_2_informative_cluster_1"
df_m_2_df_0 ="min_2_informative_cluster_0"


df_m_0_df_m_1_df_0_df_1 ="min_0_min_1_informative_cluster_0_informative_cluster_1"
df_m_0_df_m_2_df_0_df_1 ="min_0_min_2_informative_cluster_0_informative_cluster_1"
df_m_1_df_m_2_df_0_df_1 ="min_1_min_2_informative_cluster_0_informative_cluster_1"



# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_m_1_df_m_2_df_0_df_1
df= pd.read_csv("../dataset/" + selected_file + ".csv")

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12458 entries, 0 to 12457
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            12458 non-null  int64  
 1   amount          12458 non-null  float64
 2   oldbalanceOrg   12458 non-null  float64
 3   newbalanceOrig  12458 non-null  float64
 4   oldbalanceDest  12458 non-null  float64
 5   newbalanceDest  12458 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 584.1 KB


In [78]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,4,181.00,181.00,0.0,0.0,0.0
1,4,2806.00,2806.00,0.0,0.0,0.0
2,4,20128.00,20128.00,0.0,0.0,0.0
3,4,1277212.77,1277212.77,0.0,0.0,0.0
4,4,35063.63,35063.63,0.0,0.0,0.0


In [79]:
from sdv.cag import SingleTableProgrammableConstraint

class TransactionBalanceLogic(SingleTableProgrammableConstraint):
    def is_valid(self, row):
        print(row)
        amount = row['amount']
        obo = row['oldbalanceOrg']
        nbo = row['newbalanceOrig']
        obd = row['oldbalanceDest']
        nbd = row['oldbalanceDest']

        # if any(v is None for v in [t, obo, nbo, obd]):
        #     return True  # skip NaNs
        return (obd >= 0) & (obo >= 0) & (nbo >= 0) & (amount>0) & (nbd >= 0)

    def transform(self, table_data):
        return table_data  # no transformation needed

    def get_updated_metadata(self, metadata):
        new_metadata = metadata.copy()
        return new_metadata

    def reverse_transform(self, table_data):
        return table_data


In [80]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata
metadata = Metadata.detect_from_dataframe(df)
synthesizer = CTGANSynthesizer(metadata)
synthesizer.add_constraints([TransactionBalanceLogic()])
synthesizer.fit(df)

I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdv\single_table\base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [81]:
data = synthesizer.sample(df.shape[0],output_file_path="../dataset/fake_" + selected_file + ".csv")

       type      amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0         3    16865.06           0.00          149.01       332945.92   
1         1    19333.29           0.00         9527.24      1072971.04   
2         1   700574.59      476057.11          292.42       815093.35   
3         4   692291.97       54685.88         2966.57         5714.13   
4         0        6.65       64191.90     17410131.24       398137.14   
...     ...         ...            ...             ...             ...   
12453     4   582639.10      430239.72         5252.18        12470.90   
12454     3        6.65           0.00        22329.67       169901.52   
12455     3        6.65       54311.90       187787.72            0.00   
12456     3    42170.39        9146.61          320.83            0.00   
12457     4  2655070.61      575395.14         4305.28        11968.14   

       newbalanceDest  
0           480676.96  
1          1000678.73  
2           976878.64  
3            27